In [ ]:
import os
import sys
import platform
import shutil
import re
import datetime
import logging
import multiprocessing
from glob import glob
import trilabtracker as tt
# import numpy as np
# import pandas as pd

from importlib import reload
for m in tt.__all__:
    eval(f'reload(tt.{m})')
reload(tt)

In [ ]:
# Video to track.
input_file   = '../raw_videos2/videos/sf_thur_7dpf_groupA_n5b_20210513_2021-05-13-142401.mkv'

# Store output in this directory, in a subdirectory named after the video.
tracking_dir = 'output'

# Load predefined settings...
settings = tt.load_filtered_settings('settings/larval-schooling-2.xlsx')['*']
# # ..or define your own.
# settings = dict( t_start = 0, t_end = -1, n_extra = 1, n_report = 100, 
#                  n_blur = 5, block_size = 31, threshold_offset = 13, 
#                  min_area = 20, max_area = 800, ideal_area = 150, 
#                  max_aspect = 15, ideal_aspect = 8, area_penalty = 0.5, 
#                  morph_transform = [], reversal_threshold = None, 
#                  bkg = dict( t_start = 0, t_end = -1, n_training_frames = 100, 
#                              contrast_factor = 5, secondary_subtraction = True,
#                              secondary_factor = 1 )
#                ) 

# Extract trial name and number of fish from the video file's path.
d,fn       = os.path.split(input_file)
fn,ext     = os.path.splitext(fn)
output_dir = os.path.join(tracking_dir,fn)
n_ind      = int(re.findall('_n(\d+)[^\d]*_',fn)[0])

# Create tracker. Compute background. Detect tank interactively.
# settings = tt.apply_filtered_settings(filtered_settings, fn)
settings.update( input_video=input_file, output_dir=output_dir, 
                 n_ind=n_ind, trial_name=fn )
if not os.path.exists(tracking_dir):
    os.mkdir(tracking_dir)
tt.reset_logging()
tt.add_log_stream(sys.stdout, overwrite=True)
logging.info(settings['input_video']+'\n')
tracker = tt.Tracker(**settings)
tracker.init_all()

# Run tracker.
tracker.track_video()

tt.reset_logging()

In [ ]:
tt.start_gui(tracker.output_dir)

In [ ]:
# import matplotlib.pyplot as plt

# trial = tt.load_trial(tracker.output_dir)
# xy = trial['data'][:,:,:2]
# for i in range(xy.shape[1]):
#     plt.plot(*xy[:,i,:2].T)